<a href="https://colab.research.google.com/github/Mayank187/StackoverflowTags/blob/main/stackoverflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle
!pip install scikit-multilearn
!pip install matplotlib --upgrade
!pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 71.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.3/98.3 kB 7.8 MB/s eta 0:00:00
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.1.0
    Uninstalling pyparsing-3.1.0:
      Successfully uninstalled pyparsing-3.1.0
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.7.1
    Uninstalling matplotlib-3.7.1:
      Successfully uninstalled matplotlib-3.7.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 3.5 MB/s eta 0:00:00


In [2]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle competitions download facebook-recruiting-iii-keyword-extraction

100% 2.89G/2.90G [00:19<00:00, 165MB/s]
100% 2.90G/2.90G [00:19<00:00, 158MB/s]


In [4]:
!unzip facebook-recruiting-iii-keyword-extraction.zip

Archive:  facebook-recruiting-iii-keyword-extraction.zip
  inflating: SampleSubmission.csv    
  inflating: Test.zip                
  inflating: Train.zip               


In [5]:
!unzip Train.zip

Archive:  Train.zip
  inflating: Train.csv               


In [6]:
!rm Train.zip
!rm Test.zip
!rm SampleSubmission.csv
!rm facebook-recruiting-iii-keyword-extraction.zip

In [7]:
def preprocess_text(text):
    # lowercasing
    text = text.lower()

    #Remove HTML code
    clean = re.compile('<.*?>')
    text = re.sub(clean, '', text)

    # Substituting multiple spaces with single space
    text = re.sub(r'\s+', ' ', text, flags=re.I)

    # tokenize the text
    words = word_tokenize(text)

    # remove stopwords and stem the words
    stop_words = set(stopwords.words('english'))
    ps = PorterStemmer()
    words = [ps.stem(w) for w in words if w not in stop_words]

    return ' '.join(words)

# Importing the Important Libraries

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk import WordNetLemmatizer
nltk.download('wordnet')
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import category_encoders as ce
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, classification_report
from nltk.stem import PorterStemmer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report, hamming_loss, accuracy_score
from skmultilearn.problem_transform import BinaryRelevance, ClassifierChain, LabelPowerset
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
MODEL_NAME = 'bert-base-uncased'

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [9]:
import warnings
warnings.filterwarnings('ignore')

In [10]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
train_data = pd.read_csv('Train.csv',nrows=80000)

In [12]:
train_data.head()

,Id,Title,Body,Tags
0,1,How to check if an uploaded file is an image w...,<p>I'd like to check if an uploaded file is an...,php image-processing file-upload upload mime-t...
1,2,How can I prevent firefox from closing when I ...,"<p>In my favorite editor (vim), I regularly us...",firefox
2,3,R Error Invalid type (list) for variable,<p>I am import matlab file and construct a dat...,r matlab machine-learning
3,4,How do I replace special characters in a URL?,"<p>This is probably very simple, but I simply ...",c# url encoding
4,5,How to modify whois contact details?,<pre><code>function modify(.......)\n{\n $mco...,php api file-get-contents


In [13]:
train_data.shape

(80000, 4)

In [14]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Id      80000 non-null  int64 
 1   Title   80000 non-null  object
 2   Body    80000 non-null  object
 3   Tags    80000 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.4+ MB


In [15]:
train_data.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Id,80000.0,NaN,NaN,NaN,40000.5,23094.155105,1.0,20000.75,40000.5,60000.25,80000.0
Title,80000,79597,What does a bare < do?,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Body,80000,79610,<p>This is actually a design question for firm...,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Tags,80000,59708,android,665,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
train_data.isna().sum()

Id       0
Title    0
Body     0
Tags     0
dtype: int64

In [17]:
train_data.dropna(inplace = True)

In [18]:
train_data['Question'] = train_data.apply(lambda row: str(row['Title']) + str(row['Body']), axis=1)
train_data.drop(['Title','Body','Id'], axis=1, inplace=True)

In [19]:
train_data.head()

,Tags,Question
0,php image-processing file-upload upload mime-t...,How to check if an uploaded file is an image w...
1,firefox,How can I prevent firefox from closing when I ...
2,r matlab machine-learning,R Error Invalid type (list) for variable<p>I a...
3,c# url encoding,How do I replace special characters in a URL?<...
4,php api file-get-contents,How to modify whois contact details?<pre><code...


In [20]:
train_data['Question'] = train_data['Question'].apply(preprocess_text)

In [21]:
train_data.head()

,Tags,Question
0,php image-processing file-upload upload mime-t...,check upload file imag without mime type ? 'd ...
1,firefox,prevent firefox close press ctrl-win favorit e...
2,r matlab machine-learning,r error invalid type ( list ) variablei import...
3,c# url encoding,"replac special charact url ? probabl simpl , s..."
4,php api file-get-contents,modifi whoi contact detail ? function modifi (...


In [ ]:
# Convert tags into a list of tags
# train_data['Tags'] = train_data['Tags'].apply(lambda x: x.split(' '))

In [22]:
# # Convert tags into a binary matrix
# mlb = MultiLabelBinarizer()
# y = mlb.fit_transform(train_data['Tags'])
cntvectorizer = CountVectorizer(tokenizer= lambda text : text.split(" "))
y = cntvectorizer.fit_transform(train_data['Tags'])

In [23]:
cntvectorizer.vocabulary_

{'php': 10897,
 'image-processing': 6802,
 'file-upload': 5110,
 'upload': 15422,
 'mime-types': 8996,
 'firefox': 5195,
 'r': 11823,
 'matlab': 8672,
 'machine-learning': 8482,
 'c#': 1868,
 'url': 15441,
 'encoding': 4498,
 'api': 787,
 'file-get-contents': 5077,
 'proxy': 11501,
 'active-directory': 209,
 'jmeter': 7533,
 'core-plot': 2921,
 'asp.net': 971,
 'windows-phone-7': 16261,
 '.net': 9,
 'javascript': 7412,
 'code-generation': 2463,
 'sql': 13679,
 'variables': 15589,
 'parameters': 10645,
 'procedure': 11368,
 'calls': 1963,
 'obfuscation': 10061,
 'reflector': 12096,
 'algorithm': 431,
 'language-agnostic': 7939,
 'random': 11903,
 'postfix': 11195,
 'migration': 8986,
 'mdaemon': 8732,
 'documentation': 4018,
 'latex3': 7989,
 'expl3': 4787,
 'windows-7': 16213,
 'url-routing': 15449,
 'conventions': 2872,
 'temporary-files': 14542,
 'wpf': 16469,
 'binding': 1498,
 'playframework': 11034,
 'minify': 9013,
 'xml': 16633,
 'hash': 6264,
 'multidimensional-array': 9388,
 '

In [24]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(train_data['Question'], y, test_size=0.2, random_state=42)

In [26]:
# Convert text data into TF-IDF features
vectorizer = TfidfVectorizer(max_features=100000)
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [27]:
print(X_train.shape)
print(y_train.shape)

(64000, 100000)
(64000, 16899)


In [28]:
classifier = OneVsRestClassifier(SGDClassifier(loss='log_loss',alpha=0.00001, penalty='l1'), n_jobs=-1)
classifier.fit(X_train,y_train)

OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-05, loss='log_loss',
                                            penalty='l1'),
                    n_jobs=-1)

In [29]:
predictions = classifier.predict(X_test)

In [30]:
print("accuracy ", metrics.accuracy_score(y_test,predictions))
print("macro f1 score ",metrics.f1_score(y_test,predictions, average='macro'))
print("micro f1 score ", metrics.f1_score(y_test, predictions, average='micro'))
print("hamming loss ", metrics.hamming_loss(y_test,predictions))

accuracy  0.0574375
macro f1 score  0.025249555568235955
micro f1 score  0.35058388105417654
hamming loss  0.00015055624593171194


In [ ]:
import pickle
with open('model.pkl','wb') as file:
  pickle.dump(classifier, file)